### Object detection using Trasfer Learnig of CNN architecture (Use CIFAR10 Dataset)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [15]:
train_dir = "cifar-10-img/cifar-10-img/train"
test_dir = "cifar-10-img/cifar-10-img/test"

image_data_generator = ImageDataGenerator(rescale=1.0/255)
train_batch_size = 5000
test_batch_size = 1000

train_generator = image_data_generator.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=train_batch_size,
    class_mode='categorical'
)

test_generator = image_data_generator.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=test_batch_size,
    class_mode='categorical'
)

Found 40079 images belonging to 10 classes.
Found 9921 images belonging to 10 classes.


In [14]:
x_train, y_train =  train_generator[0]
x_test, y_test = test_generator[0]

print(len(x_train))
print(len(x_test))

5000
1000


In [34]:
# Load VGG16 without top layers
weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(32, 32, 3))

In [35]:
for layer in base_model.layers:
    layer.trainable = False

In [36]:
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
79/79 [==============================] - 13s 146ms/step - loss: 1.9609 - accuracy: 0.2940 - val_loss: 1.6778 - val_accuracy: 0.4030
Epoch 2/10
79/79 [==============================] - 11s 136ms/step - loss: 1.5882 - accuracy: 0.4298 - val_loss: 1.5158 - val_accuracy: 0.4540
Epoch 3/10
79/79 [==============================] - 11s 140ms/step - loss: 1.4302 - accuracy: 0.4996 - val_loss: 1.4807 - val_accuracy: 0.4790
Epoch 4/10
79/79 [==============================] - 12s 153ms/step - loss: 1.3745 - accuracy: 0.5262 - val_loss: 1.4351 - val_accuracy: 0.4790
Epoch 5/10
79/79 [==============================] - 12s 149ms/step - loss: 1.2826 - accuracy: 0.5464 - val_loss: 1.4161 - val_accuracy: 0.4950
Epoch 6/10
79/79 [==============================] - 11s 145ms/step - loss: 1.2469 - accuracy: 0.5674 - val_loss: 1.4299 - val_accuracy: 0.4790
Epoch 7/10
79/79 [==============================] - 13s 165ms/step - loss: 1.1929 - accuracy: 0.5724 - val_loss: 1.3929 - val_accuracy: 0.5040

In [38]:
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(32, 32, 3))
# freeze all layers first
for layer in base_model.layers:
    layer.trainable = False
# unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 4:]:
    layer.trainable = True

In [39]:
# fine-tuning hyper parameters
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# training fine tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
79/79 [==============================] - 53s 656ms/step - loss: 1.7799 - accuracy: 0.3460 - val_loss: 1.3549 - val_accuracy: 0.5230
Epoch 2/10
79/79 [==============================] - 50s 638ms/step - loss: 1.2856 - accuracy: 0.5438 - val_loss: 1.2997 - val_accuracy: 0.5630
Epoch 3/10
79/79 [==============================] - 49s 619ms/step - loss: 1.0308 - accuracy: 0.6512 - val_loss: 1.2233 - val_accuracy: 0.5840
Epoch 4/10
79/79 [==============================] - 48s 613ms/step - loss: 0.9103 - accuracy: 0.6888 - val_loss: 1.2163 - val_accuracy: 0.6150
Epoch 5/10
79/79 [==============================] - 52s 660ms/step - loss: 0.7820 - accuracy: 0.7410 - val_loss: 1.3421 - val_accuracy: 0.5940
Epoch 6/10
79/79 [==============================] - 53s 666ms/step - loss: 0.6464 - accuracy: 0.7752 - val_loss: 1.4619 - val_accuracy: 0.5960
Epoch 7/10
79/79 [==============================] - 58s 734ms/step - loss: 0.5341 - accuracy: 0.8192 - val_loss: 1.4673 - val_accuracy: 0.6220

In [ ]:
import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)

In [ ]:
labels = list(test_generator.class_indices.keys())

In [ ]:
n = 890
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])